# Using macros to define value-objects and other simple wrappers in Julia

In Domain-Driven Design, it is common to have a large number of small types encapsulating the primitive data types and giving them meaning. For example, consider the following definition:

In [ ]:
immutable User
    firstname::UTF8String
    lastname::UTF8String
    email::UTF8String
end

It would be much more expressive to have value objects like `Name` and `Email` for those strings, to signify that they're really different kind of strings. We would like to write, say, like this instead:

In [ ]:
immutable User
    firstname::Name
    lastname::Name
    email::Email
end

Making this change, we can for example define a function `isvalid(m::Email)` that won't be applicable at all to `Name` objects - that wouldn't have been possible with plain strings.

Now, we could of course just implement `Name` and `Email` as simple immutables, and define whatever methods we needed on them. But it's very practical to be able to use them *as if* they're really strings.

We can overcome this if we let them inherit from `AbstractString`:

In [ ]:
immutable Name <: AbstractString name::UTF8String end
immutable Email <: AbstractString email::UTF8String end

This *almost* does what we want. However, not everything works, as there are a few methods on `AbstractString` which other generic fallback methods expect, but which we have not implemented. For example, we need to implement `next` and `endof` to get everything working smoothly.

Doing this for both `Name` and `Email`, as well as for any other string wrappers we might want, quickly becomes very repetitive.

And now you know where this is going...

What if we could write

In [ ]:
@stringwrapper immutable Name{T<:AbstractString} <: AbstractString
    name::T
end

and get the method definitions for `next` and `endof` for free?

Well, we can!

# Enter `@stringwrapper`

Repetitive source transformations, such as taking a type definition and adding a couple of method definitions, is exactly the type of thing that Julia's macros are really good at. The following macro definition might look a little daunting at first, but it's really quite simple:

In [ ]:
macro stringwrapper(typedef::Expr)
    typedef.head == :type || error("The @stringwrapper macro is only applicable on type definitions")
    
    tname = (isa(typedef.args[2], Symbol)
        ? typedef.args[2]
        : isa(typedef.args[2].args[1], Symbol)
            ? typedef.args[2].args[1]
            : typedef.args[2].args[1].args[1])
    fname = (isa(typedef.args[3].args[2], Symbol)
        ? typedef.args[3].args[2]
        : typedef.args[3].args[2].args[1])

    quote
       $typedef

       Base.endof(t::$tname) = Base.endof(t.$fname)
       Base.next(t::$tname) = Base.next(t.$fname)
    end
end


There are two parts to this.

First, we single out the name of the type and the name of its field - all the casing and navigating through the AST in the two ternary statements is just to cover the various cases of how specificly the type is defined; for example, this handles both `@stringwrapper type Foo x end` and `@stringwrapper immutable Foo{T<:AbstractString} <: AbstractString x::T end`. In either case, we get `tname = :Foo` and `fname = :x`.

Secondly, we use this information to add the required method definitions. The resulting code for `Name` and `Email` is barely touched, but now we can implement the entire `AbstractString` interface for all our string-wrapping value objects, and still keep the expressiveness that comes from having them as separate types.

# Generalizing - wrapper types in general

Since all concrete types in Julia are also sealed (at least that's what it's called in C# when you can't subclass), the [composition over inheritance](https://en.wikipedia.org/wiki/Composition_over_inheritance) principle comes very naturally.

However, this also means that if you want to implement the same interface as the type you wrap, you quite often need to define a number of methods on your wrapper type to get all the functionality. Another prime example is `AbstractArray`, for which you need only implement [a few methods](http://docs.julialang.org/en/release-0.4/manual/interfaces/#abstract-arrays) as well as have your type inherit `AbstractArray`, to get a lot of stuff implemented as generic fallbacks. The `@stringwrapper` macro above could easily be given a sibling `@arraywrapper`, which implements forwarding methods to some or all of the methods required to properly extend `AbstractArray`.